In [1]:
from fdua_competition.vectorstore import build_vectorstore
from fdua_competition.enums import Mode, VectorStoreOption
from fdua_competition.rag import ResearchAssistant
from pprint import pprint

In [2]:
vectorstore = build_vectorstore("output_simple_test", Mode.TEST, VectorStoreOption.CHROMA)

[prepare_vectorstore] chroma: /fdua-competition/.fdua-competition/vectorstores/chroma
[add_document_to_vectorstore] skipping existing document: /fdua-competition/.fdua-competition/validation/documents/9.pdf
[add_document_to_vectorstore] skipping existing document: /fdua-competition/.fdua-competition/validation/documents/8.pdf
[add_document_to_vectorstore] skipping existing document: /fdua-competition/.fdua-competition/validation/documents/10.pdf
[add_document_to_vectorstore] skipping existing document: /fdua-competition/.fdua-competition/validation/documents/6.pdf
[add_document_to_vectorstore] skipping existing document: /fdua-competition/.fdua-competition/validation/documents/7.pdf
[add_document_to_vectorstore] skipping existing document: /fdua-competition/.fdua-competition/validation/documents/5.pdf
[add_document_to_vectorstore] skipping existing document: /fdua-competition/.fdua-competition/validation/documents/4.pdf
[add_document_to_vectorstore] skipping existing document: /fdua-co

In [3]:
assistant = ResearchAssistant(vectorstore)

In [4]:
res = assistant.invoke("電通についてまとめて")

In [5]:
pprint(res)

ResearchAssistantResponse(query='電通についてまとめて', response='電通は1901年創業の広告会社で通信と広告販売を融合し成長した。業界の近代化を推進し多様な事業展開を行っている。', reason='電通の歴史や事業展開に関する情報を集約し簡潔にまとめた。', organization_name='電通グループ', contexts=['創業者 光永星郎が日本広告会社を設立し広告専業化した。', '広告業界の発展に尽力し多様なメディアに進出した。', '電通は1901年創設で日本の広告業近代化を先導した。'])
